# Intro to Forge

Forge is a PyTorch-native framework designed for rapid experimentation and large-scale training of Reinforcement Learning (RL) algorithms with Large Language Models (LLMs). It's designed to:
- Express RL algorithms as naturally as psuedocode, while scaling seamlessly across clusters
- Support varying degrees of asynchrony - from fully synchronous/on-policy, to fully asynchronous/off-policy training
- Separate infrastructural concerns from algorithmic implementation
- Bias towards composable, reusable components that can be mixed and matched for different RL approaches

Forge is built on top of proven components:
- **[Monarch](https://github.com/meta-pytorch/monarch)** - PyTorch-native single-controller framework
- **[torchtitan](https://github.com/pytorch/torchtitan)** - PyTorch-native large-scale LLM training platform
- **[vLLM](https://github.com/vllm-project/vllm)** - A high-throughput, memory efficient inference and serving engine for LLMs

Our mission is to accelerate innovation in reinforcement learning by empowering researchers and developers to explore new RL algorithms and infrastructure techniques. Whether you're designing novel training methods or optimizing distributed systems, Forge provides a foundation to build upon.

## Brief Intro to Monarch
Before diving into Forge, we need to first establish the foundation. Forge is built on top of Monarch, PyTorch's native single-controller framework for distributed execution.

Forge builds many of its abstractions on top of Monarch, so it's worth introducing a few of its key concepts first. The following sections borrow from Monarch's Getting Started Guide (not public yet).

### Defining an Actor
At its core, Monarch uses [actors](https://en.wikipedia.org/wiki/Actor_model) as a way to create multi-machine programs. Actors are Python objects that expose a number of endpoint functions. These functions can be called by other actors in the system and their responses gathered asynchronously.

In [1]:
import asyncio
from monarch.actor import Actor, endpoint, this_proc

class Counter(Actor):
    def __init__(self, initial_value: int):
        self.value = initial_value

    @endpoint
    def increment(self) -> None:
        self.value += 1

    @endpoint
    def get_value(self) -> int:
        return self.value


The decorator `@endpoint` specifies functions of the Actor that can be called remotely from other actors.

### Spawning An Actor In The Local Process

We spawn actors in the current running process like so:

In [2]:
counter: Counter = this_proc().spawn("counter", Counter, initial_value=0)

`this_proc()` is a handle to a process, giving us direct control over where an actor runs. Monarch is very literal about where things are run, so that code can be written in the most efficient way. 

### Sending A Simple Message
Once an actor is spawned, we can send messages to the actor:

In [3]:
from monarch.actor import Future

fut: Future[int] = counter.get_value.call_one()

value = await fut

print(f"Counter value: {value}")

Counter value: 0


Here we invoke the `get_value` message, returning 0, the current value of the Counter. `call_one` here is referred to as an "adverb" because it modified how results of the endpoint are handled. `call_one` just invokes a single actor and gets its value.

Notice that the return value is a `Future[int]` - the message is sent asynchronously, letting the sender do other things before it needs the reply. We can `await` on the result.

### Multiple Actors at Once
Monarch scales to thousands of machines because of its ability to broadcast a single message to many actors at once, rather than send many point-to-point messages.

Monarch expresses broadcasted communication by organizing actors into a `Mesh` - a multi-dimensional container with named dimensions. An example cluster may have dimensions `{"hosts": 32, "gpus": 8}`. To create a mesh of actors, we'll create a mesh of processes and spawn an actor on them:

In [4]:
from monarch.actor import ProcMesh, this_host

procs: ProcMesh = this_host().spawn_procs(per_host={"gpus": 8})
counters: Counter = procs.spawn("counters", Counter, 0)

### Broadcasting Messages
Now messages can be sent to all actors in the mesh:

In [5]:
await counters.increment.call()

ValueMesh({gpus=8})

Note that here, we use the `call()` adverb. You will see other adverbs in Monarch code as well:
- `call_one()` - invoke a single actor and get its value (what we saw before)
- `choose()` - randomly invoke a single actor and gets its value from within a mesh of actors
- `call()` - invoke all actors in an actor mesh, and return its values as a `ValueMesh` 
- `broadcast()` - fire-and-forget all actors in an actor mesh
- `stream()` - invoke all actors and return its values as an iterator

There's much more to cover with Monarch, but these foundations provide the building blocks needed to understand how Forge creates its RL-specific services on top of this distributed actor system.

In [6]:
await procs.stop()

## Forge Services
Forge introduces *Services* - a higher-level abstraction built on top of Monarch actors. Services handle all the operational complexity of managing distributed ActorMeshes: spawning actors across nodes, fault tolerance, load balancing, and intelligent routing.

### Creating a Forge Service
Creating a Forge service requires minimal changes to actors like we've created above. You replace your Actor base with a ForgeActor, and change how you spawn the actor:

In [7]:
from forge.controller import ForgeActor
from forge.controller.service import ServiceConfig, spawn_service, shutdown_service
from monarch.actor import endpoint


class ForgeCounter(ForgeActor):
    def __init__(self, initial_value: int):
        self.value = initial_value

    @endpoint
    def increment(self) -> int:
        self.value += 1
        return self.value

    @endpoint
    def get_value(self) -> int:
        return self.value

    @endpoint
    async def reset(self):
        self.value = 0

    @endpoint
    def fail(self):
        raise RuntimeError("I was asked to fail")


counter_service = await spawn_service(
    ServiceConfig(procs_per_replica=1, num_replicas=4),
    ForgeCounter,
    initial_value=0)

Got failure on replica 2. Error:
A remote actor call has failed.
 Traceback of where the remote call failed (most recent call last):
  File "/home/allencwang/.conda/envs/forge_test/lib/python3.10/site-packages/monarch/_src/actor/actor_mesh.py", line 844, in handle
    result = the_method(*args, **kwargs)
  File "/tmp/ipykernel_1072253/2489071177.py", line 25, in fail
RuntimeError: I was asked to fail



Here, we've created a simple "Counter service" with 4 replicas, each running on 1 process.

### Service Adverbs: Operating at the Replica Level
Services introduce new adverbs that operate at the replica level, not individual actors. Since replicas can be spawned across multiple processes, each replica is essentially an ActorMesh in Monarch terms:

In [8]:
# choose() - routes to one replica (load balanced, and which may contain multiple actors)
await counter_service.increment.choose()

# call() - runs on ALL replicas
results = await counter_service.increment.call()

print(results)

[1, 2, 1, 1]


Key distinction:
- Monarch's `choose()` picks a single actor from an `ActorMesh`
- Forge's `choose()` picks a single replica (which could be an entire `ActorMesh` of actors)

This abstraction lets you think in terms of logical compute units (replicas) rather than individual processes.

### Load Balancing in Action
Services handle load balancing:

In [9]:
await counter_service.reset.call()
print("Increment on different replicas:")
for i in range(8):
    result = await counter_service.increment.choose()
    print(f"Call {i}: Counter value = {result}")

Increment on different replicas:
Call 0: Counter value = 1
Call 1: Counter value = 1
Call 2: Counter value = 1
Call 3: Counter value = 1
Call 4: Counter value = 2
Call 5: Counter value = 2
Call 6: Counter value = 2
Call 7: Counter value = 2


Each replica maintains its own state, and requests are distributed evenly.

### Sticky Session for Stateful Operations
When you need to interact with the same replica consistently:

In [10]:
# Use sticky sessions to stay on one replica
async with counter_service.session():
    await counter_service.reset.choose()
    print(await counter_service.increment.choose())
    print(await counter_service.increment.choose())
    print(await counter_service.increment.choose())
          
    final_value = await counter_service.get_value.choose()
    print(f"Final value on this replica: {final_value}")

1
2
3
Final value on this replica: 3


Sticky sessions can be crucial for efficiency, i.e. whenever you need to maintain KV cache state across multiple turns.

### Automatic Fault Tolerance
Services automatically handle failures:

In [11]:
# This will fail on one replica
try:
    await counter_service.fail.choose()
except ValueError:
    print("Replica failed, but service continues...")

# Subsequent calls automatically route around the failed replica
result = await counter_service.increment.choose()
print(f"Still working: {result}")

# The failed replica will be automatically recovered

[0] Traceback (most recent call last):
[0]   File "/home/allencwang/.conda/envs/forge_test/lib/python3.10/site-packages/monarch/_src/actor/actor_mesh.py", line 844, in handle
[0]     result = the_method(*args, **kwargs)
[0]   File "/tmp/ipykernel_1072253/2489071177.py", line 25, in fail
[0] RuntimeError: I was asked to fail


Still working: 4


>>> Aggregated Logs (2025-09-01 12:00:34) >>>
[1 similar log lines] Traceback (most recent call last):
[1 similar log lines]   File "/home/allencwang/.conda/envs/forge_test/lib/python3.10/site-packages/monarch/_src/actor/actor_mesh.py", line 844, in handle
[1 similar log lines]     result = the_method(*args, **kwargs)
[1 similar log lines]   File "/tmp/ipykernel_1072253/2489071177.py", line 25, in fail
[1 similar log lines] RuntimeError: I was asked to fail
<<< Aggregated Logs (2025-09-01 12:00:57) <<<



Behind the scenes: Forge marks unhealthy replicas, routes traffic away from them, and spawns replacements automatically.

### Why This Matters for RL
These service abstractions solve critical RL infrastructure challenges:

1. Load balancing: Distribute rollouts across policy replicas efficiently
2. Sticky sessions: Maintain state between rollouts and their associated replicas, i.e. KV cache consistency
3. Fault tolerance: Keep training running even when individual nodes fail
4. Operational simplicity: No infrastructure code in your RL algorithms

### Performance: Control Plane vs Data Plane
One important area we haven't covered yet is how Forge separates the **control plane** (service coordination) from the **data plane** (tensor transfers). You might reasonably wonder about performance implications if all data flows through TCP in a service-based architecture.

We're actively developing **TorchStore** - our solution for high-performance tensor storage and retrieval over high-bandwidth interconnects like RDMA. This separation ensures that while Forge services handle coordination and routing, heavy tensor operations bypass the service layer entirely.

*TorchStore will be covered in detail before our official release.*


Next, we'll see how these building blocks enable elegant RL algorithm expression.

In [12]:
await shutdown_service(counter_service)

## Forge-Native Services
Now let's see the power of this abstraction in action. Forge provides service implementations of common RL components that constitute typical training workloads, for instance:
- Policy: Responsible for generating trajectories and responses
- Trainer: Responsible for updating policy weights
- Reference Model: Responsible for computing reference logprobs to prevent policy drift
- Reward: Responsible for evaluating trajectory quality
- Dataset: Responsible for serving prompts and target answers
- Advantage: Responsible for computing advantages from trajectories


### Building a Synchronous RL Workflow
Let's demonstrate by building a simple on-policy RL workflow. We'll start by spinning up multiple services using a small Qwen model:

In [13]:
from forge.actors.policy import Policy, PolicyConfig, SamplingOverrides, WorkerConfig
from forge.actors.replay_buffer import ReplayBuffer
from forge.controller.actor import ForgeActor
from forge.controller.service import ServiceConfig, shutdown_service, spawn_service
from forge.data.rewards import MathReward, ThinkingReward
from apps.grpo.main import Trainer, RewardActor, ComputeAdvantages, RefModel, DatasetActor, Group, Episode


model = "Qwen/Qwen3-1.7B"
group_size = 1

(
    dataloader,
    policy,
    trainer,
    replay_buffer,
    compute_advantages,
    ref_model,
    reward_actor,
) = await asyncio.gather(
        spawn_service(
            ServiceConfig(procs_per_replica=1, num_replicas=1),
            DatasetActor,
            path="openai/gsm8k",
            config_name="main",
            split="train",
            streaming=True,
        ),
        spawn_service(
            ServiceConfig(procs_per_replica=1, with_gpus=True, num_replicas=1),
            Policy,
            config=PolicyConfig(
                worker_params=WorkerConfig(model=model),
                sampling_params=SamplingOverrides(
                    num_samples=group_size, max_tokens=16
                ),
            ),
        ),
        spawn_service(
            ServiceConfig(procs_per_replica=1, with_gpus=True, num_replicas=1),
            Trainer,
            learning_rate=1e-5,
            beta=0.1,
            model_name=model,
        ),
        spawn_service(
            ServiceConfig(procs_per_replica=1, num_replicas=1),
            ReplayBuffer,
            batch_size=2,
            max_policy_age=1,
        ),
        spawn_service(
            ServiceConfig(procs_per_replica=1, num_replicas=1),
            ComputeAdvantages,
            gamma=0.99,
            lambda_=0.95,
        ),
        spawn_service(
            ServiceConfig(procs_per_replica=1, num_replicas=1, with_gpus=True),
            RefModel,
            model_name=model,
        ),
        spawn_service(
            ServiceConfig(procs_per_replica=1, num_replicas=1),
            RewardActor,
            reward_functions=[MathReward(), ThinkingReward()],
        ))
    

INFO 09-01 12:01:04 [__init__.py:235] Automatically detected platform cuda.
[0] INFO 09-01 12:01:11 [__init__.py:235] Automatically detected platform cuda.
[0] INFO 09-01 12:01:11 [__init__.py:235] Automatically detected platform cuda.
[0] INFO 09-01 12:01:12 [__init__.py:235] Automatically detected platform cuda.
[0] INFO 09-01 12:01:12 [__init__.py:235] Automatically detected platform cuda.
[0] INFO 09-01 12:01:12 [__init__.py:235] Automatically detected platform cuda.
>>> Aggregated Logs (2025-09-01 12:01:09) >>>
[1 similar log lines] INFO 09-01 12:01:12 [__init__.py:235] Automatically detected platform cuda.
<<< Aggregated Logs (2025-09-01 12:01:12) <<<

>>> Aggregated Logs (2025-09-01 12:01:09) >>>
[1 similar log lines] INFO 09-01 12:01:11 [__init__.py:235] Automatically detected platform cuda.
<<< Aggregated Logs (2025-09-01 12:01:12) <<<

>>> Aggregated Logs (2025-09-01 12:01:09) >>>
[1 similar log lines] INFO 09-01 12:01:12 [__init__.py:235] Automatically detected platform cuda

[0] `torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 44.32it/s]
[0] `torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 43.89it/s]
>>> Aggregated Logs (2025-09-01 12:01:12) >>>
[1 similar log lines] `torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 44.32it/s]
<<< Aggregated Logs (2025-09-01 12:01:18) <<<

>>> Aggregated Logs (2025-09-01 12:01:12) >>>
[1 similar log lines] `torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 43.89it/s]
<<< Aggregated Logs (2025-09-01 12:01:18) <<<



[0] [RefModel-0/1] 2025-09-01 12:01:20 INFO Model initialized on cuda
[0] [Trainer-0/1] 2025-09-01 12:01:20 INFO Model initialized on cuda


[0] `torch_dtype` is deprecated! Use `dtype` instead!


[0] INFO 09-01 12:01:21 [config.py:1604] Using max model len 40960
>>> Aggregated Logs (2025-09-01 12:01:12) >>>
[1 similar log lines] INFO 09-01 12:01:21 [config.py:1604] Using max model len 40960
<<< Aggregated Logs (2025-09-01 12:01:21) <<<

>>> Aggregated Logs (2025-09-01 12:01:15) >>>
[1 similar log lines] [Trainer-0/1] 2025-09-01 12:01:20 INFO Model initialized on cuda
<<< Aggregated Logs (2025-09-01 12:01:21) <<<

>>> Aggregated Logs (2025-09-01 12:01:15) >>>
[1 similar log lines] [RefModel-0/1] 2025-09-01 12:01:20 INFO Model initialized on cuda
<<< Aggregated Logs (2025-09-01 12:01:21) <<<

[0] INFO 09-01 12:01:22 [config.py:2434] Chunked prefill is enabled with max_num_batched_tokens=16384.
[0] WARNING 09-01 12:01:22 [multiproc_worker_utils.py:307] Reducing Torch parallelism from 112 threads to 1 to avoid unnecessary CPU contention. Set OMP_NUM_THREADS in the external environment to tune this value as needed.
>>> Aggregated Logs (2025-09-01 12:01:21) >>>
[1 similar log lines] 

...

>>> Aggregated Logs (2025-09-01 12:01:27) >>>
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:00<00:00,  2.79it/s]
[1 similar log lines] 
<<< Aggregated Logs (2025-09-01 12:01:30) <<<



[0] INFO 09-01 12:01:34 [backends.py:530] Using cache directory: /home/allencwang/.cache/vllm/torch_compile_cache/69cb0164d6/rank_0_0/backbone for vLLM's torch.compile
>>> Aggregated Logs (2025-09-01 12:01:30) >>>
[1 similar log lines] INFO 09-01 12:01:34 [backends.py:530] Using cache directory: /home/allencwang/.cache/vllm/torch_compile_cache/69cb0164d6/rank_0_0/backbone for vLLM's torch.compile
<<< Aggregated Logs (2025-09-01 12:01:34) <<<

[0] INFO 09-01 12:01:34 [backends.py:541] Dynamo bytecode transform time: 5.20 s
[0] INFO 09-01 12:01:37 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 2.328 s
>>> Aggregated Logs (2025-09-01 12:01:34) >>>
[1 similar log lines] INFO 09-01 12:01:34 [backends.py:541] Dynamo bytecode transform time: 5.20 s
[1 similar log lines] INFO 09-01 12:01:37 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 2.328 s
<<< Aggregated Logs (2025-09-01 12:01:37) <<<

[0] INFO 09-

Capturing CUDA graph shapes: 100%|██████████| 67/67 [00:02<00:00, 33.36it/s]


[0] INFO 09-01 12:01:48 [gpu_model_runner.py:2485] Graph capturing finished in 2 secs, took 0.60 GiB
>>> Aggregated Logs (2025-09-01 12:01:47) >>>
[1 similar log lines] INFO 09-01 12:01:48 [gpu_model_runner.py:2485] Graph capturing finished in 2 secs, took 0.60 GiB
<<< Aggregated Logs (2025-09-01 12:01:50) <<<



>>> Aggregated Logs (2025-09-01 12:01:30) >>>
Capturing CUDA graph shapes: 100%|██████████| 67/67 [00:02<00:00, 33.36it/s]
<<< Aggregated Logs (2025-09-01 12:01:50) <<<



What's happening here:
- Each service is spawned independently with its own configuration
- GPU services like the `policy`, `trainer`, and `ref_model` get GPU allocation
- All services run concurrently and can be scaled independently
- The same model is used for policy and reference, but they're separate services

Notice what we're not doing:
- Managing CUDA placement across services
- Coordinating distributed training setup
- Handling inter-service communication protocols
- Writing fault tolerance and retry logic

All of this is handled by our Service abstraction.

Let's check that the services indeed work as expected:

In [14]:
prompt = "What is 3 + 5?"
responses = await policy.generate.choose(prompt=prompt)

print(responses)

[CompletionOutput(index=0, text=' Also, can you explain how to solve 12 divided by 4,', token_ids=[7281, 11, 646, 498, 10339, 1246, 311, 11625, 220, 16, 17, 17779, 553, 220, 19, 11], cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)]


The response quality isn't great (it's only a 1.7B model!), but the service infrastructure is working perfectly.

## Building the RL Training Loop
### The Role of RL in Post-Training
One way to interpret the role of RL in post-training is to align a base pre-trained model towards hard-to-define targets. The goal is "sampling" the right data that we think will best align the model.

This is the role of "rollouts" - creating the dataset used to update our policy. Rather than training on a static dataset, RL dynamically generates training data by having the current policy interact with the environment.

Let's build a step-by-step synchronous training loop to see how these services work together. The basic RL cycle is:

1. Collect Experience: Get a prompt, generate a response, evaluate the reward
2. Compute Advantages: Calculate how much better/worse each action was than expected
3. Store Experience: Add the episode to our replay buffer
4. Sample & Train: Get a batch of experiences and update the policy
5. Repeat: Continue this cycle to improve the policy


In [15]:
from apps.grpo.main import Episode, Group


async def simple_rl_step():
    """Execute one complete RL training step"""
    
    # ===== Generate a rollout =====
    sample = await dataloader.__next__.choose()
    prompt, target = sample["question"], sample["answer"]
    
    print(f"Prompt: {prompt}")
    print(f"Target: {target}")
    
    actions = await policy.generate.choose(prompt=prompt)
    print(f"Policy response: {actions[0].text}")
    
    ref_logprobs = await ref_model.forward.choose(actions[0].token_ids)    
    reward = await reward_actor.evaluate_response.choose(
        prompt=prompt, 
        response=actions[0].text, 
        target=target
    )
    print(f"Reward: {reward}")
    
    episode = Episode(
        episode_id=0,
        prompt=prompt,
        target=target, 
        policy_version=0,
    )
    
    episode.add_group(Group(
        response=actions[0].text,
        ref_logprobs=ref_logprobs,
        reward=reward,
    ))
    
    advantages = await compute_advantages.__call__.choose(episode.groups)
    episode.groups[0].advantage = advantages[0]
    print(f"Advantage: {advantages[0]}")    
    await replay_buffer.add.choose(episode)
    print("Episode stored in replay buffer")
    
    # ===== Train on the batch ===== 
    batch = await replay_buffer.sample.choose(curr_policy_version=0)
    if batch is not None:
        print("Training on batch...")
        training_result = await trainer.train_step.choose(batch)
        loss = training_result.get("loss", 0.0)
        print(f"Training loss: {loss}")
        return loss
    else:
        print("Not enough data in buffer yet")
        return None


for step in range(10):
    print(f"\n--- RL Step {step + 1} ---")
    loss = await simple_rl_step()
    if loss:
        print(f"Step {step + 1} complete, loss: {loss:.4f}")
    else:
        print(f"Step {step + 1} complete, building buffer...")


--- RL Step 1 ---
Prompt: Karen’s work tote bag is twice the weight of her husband Kevin’s briefcase when the briefcase is empty. When Kevin puts his laptop and work papers in his briefcase, it is twice the weight of Karen’s tote. Kevin’s work papers are a sixth of the weight of the contents of his full briefcase. If Karen’s tote weighs 8 pounds, how many more pounds does Kevin’s laptop weigh than Karen’s tote?
Target: 2
Policy response:  To solve this problem, let's break it down step by step.

First,
Reward: 0.1
Advantage: 0.0
[0] INFO 09-01 12:02:14 [__init__.py:235] Automatically detected platform cuda.
>>> Aggregated Logs (2025-09-01 12:01:09) >>>
[1 similar log lines] INFO 09-01 12:02:14 [__init__.py:235] Automatically detected platform cuda.
<<< Aggregated Logs (2025-09-01 12:02:14) <<<

Episode stored in replay buffer
Not enough data in buffer yet
⏳ Step 1 complete, building buffer...

--- RL Step 2 ---
Prompt: A deep-sea monster rises from the waters once every hundred years 

**Note**: The model responses aren't great (1.7B parameters + 16 token limit = not exactly o1!), but notice how clean the RL algorithm code is. The power of these abstractions is that you can focus on the algorithm logic while all the distributed coordination happens automatically behind the scenes.

TODO - conclude this with trainer->inference weight sync and demonstrate how the response changes

## Next Steps
This simple example demonstrates the core concepts, but for a production-ready implementation, check out our full GRPO (Group Relative Policy Optimization) example at apps/grpo/main.py. It includes the complete async training loops, proper logging, model weight synchronization, and all the optimizations needed for large-scale RL training.

In [16]:
await asyncio.gather(
    shutdown_service(policy),
    shutdown_service(trainer),
    shutdown_service(replay_buffer),
    shutdown_service(dataloader),
    shutdown_service(compute_advantages),
    shutdown_service(ref_model),
    shutdown_service(reward_actor))

[0] /home/allencwang/.conda/envs/forge_test/lib/python3.10/multiprocessing/resource_tracker.py:224: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
[0]   warnings.warn('resource_tracker: There appear to be %d '
[0] /home/allencwang/.conda/envs/forge_test/lib/python3.10/multiprocessing/resource_tracker.py:224: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
[0]   warnings.warn('resource_tracker: There appear to be %d '
[0] /home/allencwang/.conda/envs/forge_test/lib/python3.10/multiprocessing/resource_tracker.py:224: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
[0]   warnings.warn('resource_tracker: There appear to be %d '
[0] /home/allencwang/.conda/envs/forge_test/lib/python3.10/multiprocessing/resource_tracker.py:224: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
[0]   w

[None, None, None, None, None, None, None]